#                 Initialize

In [4]:
import pandas as pd
import os
import numpy as np
# from IPython.display import display, Markdown

# Import the required ArcGIS API for Python modules
import arcgis
from arcgis.gis import GIS
gis = arcgis.GIS()

In [5]:
# directories
working_directory       = os.getcwd()
data_folder             = os.path.join(working_directory, "data"                         )
intermediate_folder     = os.path.join(working_directory, "intermediate"                 )
results_folder          = os.path.join(working_directory, "results"                      )
webapplayers_folder     = os.path.join(results_folder   , r"webapplayers"                )
webapplayersshps_folder = os.path.join(results_folder   , r"webapplayers\shapefiles"     )
webapplayerszips_folder = os.path.join(results_folder   , r"webapplayers\shapefiles_zips")

centers_smalldist_shp = os.path.join(data_folder, 'WasatchChoiceCenters_WFSmallAnalysisDists_v2\\WasatchChoiceCenters_WFSmallAnalysisDists_v2.shp')

csvCentersSmallDist   = os.path.join(data_folder, "centers_small_districts_od_by_mode.csv")
dfCentersSmallDistOD  = pd.read_csv(csvCentersSmallDist)

dfModes = pd.DataFrame({'mode'      :['ALL','RAI' ,'BUS','BIC'    ,'PED'       ,'TRU'  ],
                        'mode_label':['All','Rail','Bus','Bicycle','Pedestrian','Truck']})
display(dfModes)

# big data field names
#fnCounts = 'o_d_traffic_sample_trip_counts'
fnVolume = 'odvolume'

# show numbers with commas
pd.options.display.float_format = '{:,.0f}'.format

,mode,mode_label
0,ALL,All
1,RAI,Rail
2,BUS,Bus
3,BIC,Bicycle
4,PED,Pedestrian
5,TRU,Truck


In [6]:
dfDateRanges = dfCentersSmallDistOD.groupby(['date_range'],as_index=False).agg(NUMREC=('OID','count'))
dfDateRanges['year'  ] = dfDateRanges['date_range'].str[:4 ].astype(int)
dfDateRanges['season'] = dfDateRanges['date_range'].str[-2:]
display(dfDateRanges)
dfDateRanges.to_csv(os.path.join(intermediate_folder,'dateranges.csv'), index=False)

dfYears = dfDateRanges.groupby(['year'],as_index=False).agg(NUMREC=('NUMREC','count'))
dfYears['value'] = dfYears['year']
dfYears['label'] = dfYears['year']
display(dfYears)
dfYears.to_csv(os.path.join(intermediate_folder,'years.csv'), index=False)

dfSeasons = pd.DataFrame({'value':['YR'         ,'01'     ,'02'      ,'03'   ,'04'   ,'05' ,'06'  ,'07'  ,'08'    ,'09'       ,'10'     ,'11'      ,'12'      ,'BM'         ],
                          'label':['Entire Year','January','February','March','April','May','June','July','August','September','October','November','December','Bike Months']})
display(dfSeasons)
dfSeasons.to_csv(os.path.join(intermediate_folder,'seasons.csv'), index=False)


,date_range,NUMREC,year,season
0,2019_01,281528,2019,01
1,2019_02,278946,2019,02
2,2019_03,295986,2019,03
3,2019_04,337322,2019,04
4,2019_05,341743,2019,05
5,2019_06,314328,2019,06
6,2019_07,312599,2019,07
7,2019_08,315693,2019,08
8,2019_09,347835,2019,09
9,2019_10,343787,2019,10


,year,NUMREC,value,label
0,2019,14,2019,2019
1,2020,14,2020,2020
2,2021,14,2021,2021


,value,label
0,YR,Entire Year
1,01,January
2,02,February
3,03,March
4,04,April
5,05,May
6,06,June
7,07,July
8,08,August
9,09,September


In [7]:
sdfCentersSmallDist = pd.DataFrame.spatial.from_featureclass(centers_smalldist_shp)
#sdfCentersSmallDist
dfCSD = sdfCentersSmallDist[['id','group']]
dfCSD

,id,group
0,11,smalldist
1,21,smalldist
2,31,smalldist
3,32,smalldist
4,41,smalldist
...,...,...
133,"2,010",center
134,"2,011",center
135,"2,012",center
136,"2,013",center


In [8]:
dfCenterJson = sdfCentersSmallDist[sdfCentersSmallDist['group']=='center'].copy()
dfCenterJson['value'] = dfCenterJson['id'].astype(int)
dfCenterJson['label'] = dfCenterJson['name']
dfCenterJson = dfCenterJson[['value','label']]
display(dfCenterJson)
dfCenterJson.to_json(os.path.join('results',r'data\centers.json'),orient='records')


,value,label
123,2000,Downtown Provo
124,2001,Thanksgiving Point
125,2002,Layton Midtown
126,2003,The District
127,2004,9th and 9th Center
128,2005,Sugarhouse
129,2006,Murray Downtown
130,2007,The Cairns
131,2008,West Jordan City Center
132,2009,Fairbourne Station


In [9]:
dfCentersSmallDistOD_wOGroups = pd.DataFrame.merge(dfCentersSmallDistOD,dfCSD,left_on='OID',right_on='id')
dfCentersSmallDistOD_wOGroups = dfCentersSmallDistOD_wOGroups.drop(columns='id')
dfCentersSmallDistOD_wOGroups = dfCentersSmallDistOD_wOGroups.rename(columns={'group':'ogroup'})
dfCentersSmallDistOD_wOGroups

,OID,DID,day_type,day_part,odvolume,odtime,mode,date_range,ogroup
0,101,101,0,0,15720,770,TRU,2019_01,smalldist
1,101,101,0,1,984,812,TRU,2019_01,smalldist
2,101,101,0,2,3093,881,TRU,2019_01,smalldist
3,101,101,0,3,7330,743,TRU,2019_01,smalldist
4,101,101,0,4,3056,703,TRU,2019_01,smalldist
...,...,...,...,...,...,...,...,...,...
12924530,662,662,2,5,2,NaN,ALL,2021_YR,smalldist
12924531,662,721,0,0,0,NaN,ALL,2021_YR,smalldist
12924532,662,721,0,3,0,NaN,ALL,2021_YR,smalldist
12924533,662,721,1,0,0,NaN,ALL,2021_YR,smalldist


In [10]:
dfCentersSmallDistOD_wOGroupsDGroups = pd.DataFrame.merge(dfCentersSmallDistOD_wOGroups,dfCSD,left_on='DID',right_on='id')
dfCentersSmallDistOD_wOGroupsDGroups = dfCentersSmallDistOD_wOGroupsDGroups.drop(columns='id')
dfCentersSmallDistOD_wOGroupsDGroups = dfCentersSmallDistOD_wOGroupsDGroups.rename(columns={'group':'dgroup'})
dfCentersSmallDistOD_wOGroupsDGroups


,OID,DID,day_type,day_part,odvolume,odtime,mode,date_range,ogroup,dgroup
0,101,101,0,0,15720,770,TRU,2019_01,smalldist,smalldist
1,101,101,0,1,984,812,TRU,2019_01,smalldist,smalldist
2,101,101,0,2,3093,881,TRU,2019_01,smalldist,smalldist
3,101,101,0,3,7330,743,TRU,2019_01,smalldist,smalldist
4,101,101,0,4,3056,703,TRU,2019_01,smalldist,smalldist
...,...,...,...,...,...,...,...,...,...,...
12924530,662,662,2,0,15,"1,199",ALL,2021_YR,smalldist,smalldist
12924531,662,662,2,2,5,NaN,ALL,2021_YR,smalldist,smalldist
12924532,662,662,2,3,4,NaN,ALL,2021_YR,smalldist,smalldist
12924533,662,662,2,4,3,NaN,ALL,2021_YR,smalldist,smalldist


In [11]:
dfCentersSmallDistOD_wOGroupsDGroups['group'] = ''
dfCentersSmallDistOD_wOGroupsDGroups.loc[(dfCentersSmallDistOD_wOGroupsDGroups['ogroup']=='smalldist') & (dfCentersSmallDistOD_wOGroupsDGroups['dgroup']=='smalldist'),'group'] = 'smalldist'
dfCentersSmallDistOD_wOGroupsDGroups.loc[(dfCentersSmallDistOD_wOGroupsDGroups['ogroup']=='center'   ) & (dfCentersSmallDistOD_wOGroupsDGroups['dgroup']=='smalldist'),'group'] = 'centertosmalldist'
dfCentersSmallDistOD_wOGroupsDGroups.loc[(dfCentersSmallDistOD_wOGroupsDGroups['ogroup']=='smalldist') & (dfCentersSmallDistOD_wOGroupsDGroups['dgroup']=='center'   ),'group'] = 'centertosmalldist'
dfCentersSmallDistOD_wGroups = dfCentersSmallDistOD_wOGroupsDGroups.drop(columns=['ogroup','dgroup'])

# only include rows with groups
dfCentersSmallDistOD_wGroups = dfCentersSmallDistOD_wGroups[dfCentersSmallDistOD_wGroups['group']!='']
dfCentersSmallDistOD_wGroups

,OID,DID,day_type,day_part,odvolume,odtime,mode,date_range,group
0,101,101,0,0,15720,770,TRU,2019_01,smalldist
1,101,101,0,1,984,812,TRU,2019_01,smalldist
2,101,101,0,2,3093,881,TRU,2019_01,smalldist
3,101,101,0,3,7330,743,TRU,2019_01,smalldist
4,101,101,0,4,3056,703,TRU,2019_01,smalldist
...,...,...,...,...,...,...,...,...,...
12924530,662,662,2,0,15,"1,199",ALL,2021_YR,smalldist
12924531,662,662,2,2,5,NaN,ALL,2021_YR,smalldist
12924532,662,662,2,3,4,NaN,ALL,2021_YR,smalldist
12924533,662,662,2,4,3,NaN,ALL,2021_YR,smalldist


In [12]:
dfCentersSmallDistOD_wGroups[dfCentersSmallDistOD_wGroups['group']=='centertosmalldist']

,OID,DID,day_type,day_part,odvolume,odtime,mode,date_range,group
26377,2000,101,0,0,147,NaN,TRU,2019_01,centertosmalldist
26378,2000,101,0,1,11,NaN,TRU,2019_01,centertosmalldist
26379,2000,101,0,4,84,NaN,TRU,2019_01,centertosmalldist
26380,2000,101,0,5,53,NaN,TRU,2019_01,centertosmalldist
26381,2000,101,1,0,189,NaN,TRU,2019_01,centertosmalldist
...,...,...,...,...,...,...,...,...,...
12920268,2013,662,1,2,0,NaN,ALL,2020_BM,centertosmalldist
12920269,2013,662,0,0,0,NaN,ALL,2020_YR,centertosmalldist
12920270,2013,662,0,2,0,NaN,ALL,2020_YR,centertosmalldist
12920271,2013,662,1,0,0,NaN,ALL,2020_YR,centertosmalldist


In [13]:
dfCentersSmallDistOD_wGroups.groupby('group').agg(NUMREC=('OID','count'))

,NUMREC
group,
centertosmalldist,2495754
smalldist,10243731


In [14]:
# export intermediate file
dfCentersSmallDistOD_wGroups.to_csv(os.path.join(intermediate_folder,'od_by_mode_with_groups.csv'),index=False)

## Create Percent Distribution

In [32]:
# remove intrazonals from analysis
dfOD_noIntrazonals = dfCentersSmallDistOD_wGroups.loc[dfCentersSmallDistOD_wGroups['OID']!=dfCentersSmallDistOD_wGroups['DID']]
dfOD_wiIntrazonals = dfCentersSmallDistOD_wGroups

# create group totals for origin and destination and then find percent breakdown for each
dfOD_noIntrazonals_Orig_GroupTotals = dfOD_noIntrazonals.groupby(['group','OID','day_type','day_part','date_range','mode'],as_index=False).agg(OTotalVolume=(fnVolume,'sum'))
dfOD_noIntrazonals_Dest_GroupTotals = dfOD_noIntrazonals.groupby(['group','DID','day_type','day_part','date_range','mode'],as_index=False).agg(DTotalVolume=(fnVolume,'sum'))
dfOD_wiIntrazonals_Orig_GroupTotals = dfOD_wiIntrazonals.groupby(['group','OID','day_type','day_part','date_range','mode'],as_index=False).agg(OTotalVolume=(fnVolume,'sum'))
dfOD_wiIntrazonals_Dest_GroupTotals = dfOD_wiIntrazonals.groupby(['group','DID','day_type','day_part','date_range','mode'],as_index=False).agg(DTotalVolume=(fnVolume,'sum'))

dfOD_noIntrazonals_Orig_withGroupTotals = pd.DataFrame.merge(dfOD_noIntrazonals,dfOD_noIntrazonals_Orig_GroupTotals,on=('group','OID','day_type','day_part','date_range','mode'))
dfOD_noIntrazonals_Dest_withGroupTotals = pd.DataFrame.merge(dfOD_noIntrazonals,dfOD_noIntrazonals_Dest_GroupTotals,on=('group','DID','day_type','day_part','date_range','mode'))
dfOD_wiIntrazonals_Orig_withGroupTotals = pd.DataFrame.merge(dfOD_wiIntrazonals,dfOD_wiIntrazonals_Orig_GroupTotals,on=('group','OID','day_type','day_part','date_range','mode'))
dfOD_wiIntrazonals_Dest_withGroupTotals = pd.DataFrame.merge(dfOD_wiIntrazonals,dfOD_wiIntrazonals_Dest_GroupTotals,on=('group','DID','day_type','day_part','date_range','mode'))

dfOD_noIntrazonals_Orig_withGroupTotals['OPercentTotal'] = round(dfOD_noIntrazonals_Orig_withGroupTotals[fnVolume] / dfOD_noIntrazonals_Orig_withGroupTotals['OTotalVolume'],4)
dfOD_noIntrazonals_Dest_withGroupTotals['DPercentTotal'] = round(dfOD_noIntrazonals_Dest_withGroupTotals[fnVolume] / dfOD_noIntrazonals_Dest_withGroupTotals['DTotalVolume'],4)
dfOD_wiIntrazonals_Orig_withGroupTotals['OPercentTotal'] = round(dfOD_wiIntrazonals_Orig_withGroupTotals[fnVolume] / dfOD_wiIntrazonals_Orig_withGroupTotals['OTotalVolume'],4)
dfOD_wiIntrazonals_Dest_withGroupTotals['DPercentTotal'] = round(dfOD_wiIntrazonals_Dest_withGroupTotals[fnVolume] / dfOD_wiIntrazonals_Dest_withGroupTotals['DTotalVolume'],4)

dfODSummary_noInterzonals = pd.DataFrame.merge(dfOD_noIntrazonals_Orig_withGroupTotals, dfOD_noIntrazonals_Dest_withGroupTotals, on=('group','OID','DID','day_type','day_part','odvolume','odtime','mode','date_range'))
dfODSummary_wiInterzonals = pd.DataFrame.merge(dfOD_wiIntrazonals_Orig_withGroupTotals, dfOD_wiIntrazonals_Dest_withGroupTotals, on=('group','OID','DID','day_type','day_part','odvolume','odtime','mode','date_range'))

dfODSummary_noInterzonals['odvolume'] = dfODSummary_noInterzonals['odvolume'].astype(int)
dfODSummary_wiInterzonals['odvolume'] = dfODSummary_wiInterzonals['odvolume'].astype(int)

dfODSummary_noInterzonals = dfODSummary_noInterzonals.fillna(0)
dfODSummary_wiInterzonals = dfODSummary_wiInterzonals.fillna(0)


In [33]:
dfOD_wiIntrazonals_Orig_withGroupTotals[(dfOD_wiIntrazonals_Orig_withGroupTotals['OID']==21) & (dfOD_wiIntrazonals_Orig_withGroupTotals['DID']==21)]

,OID,DID,day_type,day_part,odvolume,odtime,mode,date_range,group,OTotalVolume,OPercentTotal
2216957,21,21,0,0,111,NaN,TRU,2019_01,smalldist,4000,0.0278
2217001,21,21,0,2,11,NaN,TRU,2019_01,smalldist,764,0.0144
2217029,21,21,0,3,53,NaN,TRU,2019_01,smalldist,1294,0.0410
2217061,21,21,0,4,26,NaN,TRU,2019_01,smalldist,1230,0.0211
2217091,21,21,0,5,11,NaN,TRU,2019_01,smalldist,421,0.0261
...,...,...,...,...,...,...,...,...,...,...,...
11436492,21,21,1,1,366,"1,229.0000",PED,2021_YR,smalldist,366,1.0000
11436493,21,21,1,5,687,"1,070.0000",PED,2021_YR,smalldist,706,0.9731
11436496,21,21,2,1,391,"1,183.0000",PED,2021_YR,smalldist,395,0.9899
11436498,21,21,2,4,1172,"1,128.0000",PED,2021_YR,smalldist,1196,0.9799


In [34]:
dfODSummary_noInterzonals = dfODSummary_noInterzonals[['group','OID','DID','date_range','day_type','day_part','mode','odvolume','OPercentTotal','DPercentTotal']]
dfODSummary_wiInterzonals = dfODSummary_wiInterzonals[['group','OID','DID','date_range','day_type','day_part','mode','odvolume','OPercentTotal','DPercentTotal']]

# show numbers with commas
pd.options.display.float_format = '{:,.8f}'.format

display(dfOD_noIntrazonals_Orig_withGroupTotals)
display(dfOD_noIntrazonals_Dest_withGroupTotals)
display(dfODSummary_noInterzonals)
display(dfOD_wiIntrazonals_Orig_withGroupTotals)
display(dfOD_wiIntrazonals_Dest_withGroupTotals)
display(dfODSummary_wiInterzonals)

# show numbers with commas
pd.options.display.float_format = '{:,.0f}'.format

,OID,DID,day_type,day_part,odvolume,odtime,mode,date_range,group,OTotalVolume,OPercentTotal
0,102,101,0,0,1991,758.00000000,TRU,2019_01,smalldist,7279,0.27350000
1,102,11,0,0,5,NaN,TRU,2019_01,smalldist,7279,0.00070000
2,102,111,0,0,730,"1,345.00000000",TRU,2019_01,smalldist,7279,0.10030000
3,102,112,0,0,58,NaN,TRU,2019_01,smalldist,7279,0.00800000
4,102,121,0,0,1056,"1,019.00000000",TRU,2019_01,smalldist,7279,0.14510000
...,...,...,...,...,...,...,...,...,...,...,...
12401790,661,662,0,3,2,"1,453.00000000",BIC,2019_10,smalldist,2,1.00000000
12401791,661,662,0,0,0,"1,453.00000000",BIC,2019_BM,smalldist,0,NaN
12401792,661,662,0,3,0,"1,453.00000000",BIC,2019_BM,smalldist,0,NaN
12401793,661,662,0,0,0,"1,453.00000000",BIC,2019_YR,smalldist,0,NaN


,OID,DID,day_type,day_part,odvolume,odtime,mode,date_range,group,DTotalVolume,DPercentTotal
0,102,101,0,0,1991,758.00000000,TRU,2019_01,smalldist,44973,0.04430000
1,11,101,0,0,74,NaN,TRU,2019_01,smalldist,44973,0.00160000
2,111,101,0,0,2533,"1,312.00000000",TRU,2019_01,smalldist,44973,0.05630000
3,112,101,0,0,147,"2,016.00000000",TRU,2019_01,smalldist,44973,0.00330000
4,121,101,0,0,7644,"1,209.00000000",TRU,2019_01,smalldist,44973,0.17000000
...,...,...,...,...,...,...,...,...,...,...,...
12401790,621,662,1,5,3,278.00000000,PED,2021_YR,smalldist,3,1.00000000
12401791,621,662,2,4,7,938.00000000,PED,2021_YR,smalldist,7,1.00000000
12401792,631,662,1,3,6,NaN,ALL,2020_10,smalldist,6,1.00000000
12401793,691,662,1,3,2,NaN,ALL,2020_11,smalldist,2,1.00000000


,group,OID,DID,date_range,day_type,day_part,mode,odvolume,OPercentTotal,DPercentTotal
0,smalldist,102,101,2019_01,0,0,TRU,1991,0.27350000,0.04430000
1,smalldist,102,11,2019_01,0,0,TRU,5,0.00070000,0.00170000
2,smalldist,102,111,2019_01,0,0,TRU,730,0.10030000,0.02400000
3,smalldist,102,112,2019_01,0,0,TRU,58,0.00800000,0.00990000
4,smalldist,102,121,2019_01,0,0,TRU,1056,0.14510000,0.02140000
...,...,...,...,...,...,...,...,...,...,...
12401790,smalldist,661,662,2019_10,0,3,BIC,2,1.00000000,1.00000000
12401791,smalldist,661,662,2019_BM,0,0,BIC,0,0.00000000,0.00000000
12401792,smalldist,661,662,2019_BM,0,3,BIC,0,0.00000000,0.00000000
12401793,smalldist,661,662,2019_YR,0,0,BIC,0,0.00000000,0.00000000


,OID,DID,day_type,day_part,odvolume,odtime,mode,date_range,group,OTotalVolume,OPercentTotal
0,101,101,0,0,15720,770.00000000,TRU,2019_01,smalldist,63617,0.24710000
1,101,102,0,0,1771,733.00000000,TRU,2019_01,smalldist,63617,0.02780000
2,101,11,0,0,168,NaN,TRU,2019_01,smalldist,63617,0.00260000
3,101,111,0,0,2224,"1,349.00000000",TRU,2019_01,smalldist,63617,0.03500000
4,101,112,0,0,147,"1,573.00000000",TRU,2019_01,smalldist,63617,0.00230000
...,...,...,...,...,...,...,...,...,...,...,...
12739480,662,662,1,2,2,480.00000000,PED,2021_YR,smalldist,2,1.00000000
12739481,662,662,1,5,7,170.00000000,PED,2021_YR,smalldist,7,1.00000000
12739482,662,662,2,1,4,671.00000000,PED,2021_YR,smalldist,4,1.00000000
12739483,662,662,2,3,72,"1,005.00000000",PED,2021_YR,smalldist,72,1.00000000


,OID,DID,day_type,day_part,odvolume,odtime,mode,date_range,group,DTotalVolume,DPercentTotal
0,101,101,0,0,15720,770.00000000,TRU,2019_01,smalldist,60693,0.25900000
1,102,101,0,0,1991,758.00000000,TRU,2019_01,smalldist,60693,0.03280000
2,11,101,0,0,74,NaN,TRU,2019_01,smalldist,60693,0.00120000
3,111,101,0,0,2533,"1,312.00000000",TRU,2019_01,smalldist,60693,0.04170000
4,112,101,0,0,147,"2,016.00000000",TRU,2019_01,smalldist,60693,0.00240000
...,...,...,...,...,...,...,...,...,...,...,...
12739480,662,662,2,4,10,"1,826.00000000",PED,2021_BM,smalldist,10,1.00000000
12739481,662,662,0,1,1,671.00000000,PED,2021_YR,smalldist,1,1.00000000
12739482,662,662,1,4,9,647.00000000,PED,2021_YR,smalldist,9,1.00000000
12739483,662,662,2,1,4,671.00000000,PED,2021_YR,smalldist,4,1.00000000


,group,OID,DID,date_range,day_type,day_part,mode,odvolume,OPercentTotal,DPercentTotal
0,smalldist,101,101,2019_01,0,0,TRU,15720,0.24710000,0.25900000
1,smalldist,101,102,2019_01,0,0,TRU,1771,0.02780000,0.20660000
2,smalldist,101,11,2019_01,0,0,TRU,168,0.00260000,0.05570000
3,smalldist,101,111,2019_01,0,0,TRU,2224,0.03500000,0.05440000
4,smalldist,101,112,2019_01,0,0,TRU,147,0.00230000,0.02200000
...,...,...,...,...,...,...,...,...,...,...
12739480,smalldist,662,662,2021_YR,1,2,PED,2,1.00000000,0.25000000
12739481,smalldist,662,662,2021_YR,1,5,PED,7,1.00000000,0.70000000
12739482,smalldist,662,662,2021_YR,2,1,PED,4,1.00000000,1.00000000
12739483,smalldist,662,662,2021_YR,2,3,PED,72,1.00000000,0.88890000


In [35]:
dfX = dfODSummary_noInterzonals

dfX_pivotMode_Vol = dfX.pivot_table(index=('group','OID','DID','date_range','day_type','day_part'),values=('odvolume'),columns='mode')
dfX_pivotMode_Vol = dfX_pivotMode_Vol.reset_index()
dfX_pivotMode_Vol = dfX_pivotMode_Vol.fillna(0)
dfX_pivotMode_Vol = dfX_pivotMode_Vol.rename(columns={'OID':'O','DID':'D','ALL':"AV",'BIC':'KV','BUS':'SV','PED':'PV','RAI':'RV','TRU':'TV'})

dfX_pivotMode_Vol['AV'] = dfX_pivotMode_Vol['AV'].astype(int)
dfX_pivotMode_Vol['KV'] = dfX_pivotMode_Vol['KV'].astype(int)
dfX_pivotMode_Vol['SV'] = dfX_pivotMode_Vol['SV'].astype(int)
dfX_pivotMode_Vol['PV'] = dfX_pivotMode_Vol['PV'].astype(int)
dfX_pivotMode_Vol['RV'] = dfX_pivotMode_Vol['RV'].astype(int)
dfX_pivotMode_Vol['TV'] = dfX_pivotMode_Vol['TV'].astype(int)

display(dfX_pivotMode_Vol)

# show numbers with commas
pd.options.display.float_format = '{:,.4f}'.format

dfX_pivotMode_OPct = dfX.pivot_table(index=('group','OID','DID','date_range','day_type','day_part'),values=('OPercentTotal'),columns='mode')
dfX_pivotMode_OPct = dfX_pivotMode_OPct.reset_index()
dfX_pivotMode_OPct = dfX_pivotMode_OPct.fillna(0)
dfX_pivotMode_OPct = dfX_pivotMode_OPct.rename(columns={'OID':'O','DID':'D','ALL':"AO",'BIC':'KO','BUS':'SO','PED':'PO','RAI':'RO','TRU':'TO'})
display(dfX_pivotMode_OPct)

dfX_pivotMode_DPct = dfX.pivot_table(index=('group','OID','DID','date_range','day_type','day_part'),values=('DPercentTotal'),columns='mode')
dfX_pivotMode_DPct = dfX_pivotMode_DPct.reset_index()
dfX_pivotMode_DPct = dfX_pivotMode_DPct.fillna(0)
dfX_pivotMode_DPct = dfX_pivotMode_DPct.rename(columns={'OID':'O','DID':'D','ALL':"AD",'BIC':'KD','BUS':'SD','PED':'PD','RAI':'RD','TRU':'TD'})
display(dfX_pivotMode_DPct)

dfX_pivotMode_Cmb = pd.DataFrame.merge(dfX_pivotMode_Vol, dfX_pivotMode_OPct, on=('group','O','D','date_range','day_type','day_part'), how='left')
dfX_pivotMode_Cmb = pd.DataFrame.merge(dfX_pivotMode_Cmb, dfX_pivotMode_DPct, on=('group','O','D','date_range','day_type','day_part'), how='left')

dfX_pivotMode_Cmb['O'] = dfX_pivotMode_Cmb['O'].apply(lambda x: '{0:0>4}'.format(x))
dfX_pivotMode_Cmb['D'] = dfX_pivotMode_Cmb['D'].apply(lambda x: '{0:0>4}'.format(x))
dfX_pivotMode_Cmb['I'] = dfX_pivotMode_Cmb['O'].astype(str) + '_' + dfX_pivotMode_Cmb['D'].astype(str)

dfX_pivotMode_Cmb = dfX_pivotMode_Cmb.drop(columns=['O','D'])

display(dfX_pivotMode_Cmb)

dfFiles = dfX_pivotMode_Cmb.groupby(['group','date_range','day_type','day_part'],as_index=False).agg(NUMREC=('I','count'))

for index, row in dfFiles.iterrows():
    #print(row['date_range'])
    dfExport = dfX_pivotMode_Cmb[(dfX_pivotMode_Cmb['date_range']==row['date_range']) & (dfX_pivotMode_Cmb['day_type']==row['day_type']) & (dfX_pivotMode_Cmb['day_part']==row['day_part'])]
    dfExport = dfExport.drop(columns=['group','date_range','day_part','day_type'])
    dfExport = dfExport.set_index('I')
    dfExport.to_json(os.path.join('results', row['group'] + '_' + str(row['date_range']) + '_' + str(row['day_type']) + '_' + str(row['day_part']) + '_noIntrazonals.json'),orient='index')

KeyboardInterrupt: 

In [36]:
dfX = dfODSummary_wiInterzonals

dfX_pivotMode_Vol = dfX.pivot_table(index=('group','OID','DID','date_range','day_type','day_part'),values=('odvolume'),columns='mode')
dfX_pivotMode_Vol = dfX_pivotMode_Vol.reset_index()
dfX_pivotMode_Vol = dfX_pivotMode_Vol.fillna(0)
dfX_pivotMode_Vol = dfX_pivotMode_Vol.rename(columns={'OID':'O','DID':'D','ALL':"AV",'BIC':'KV','BUS':'SV','PED':'PV','RAI':'RV','TRU':'TV'})

dfX_pivotMode_Vol['AV'] = dfX_pivotMode_Vol['AV'].astype(int)
dfX_pivotMode_Vol['KV'] = dfX_pivotMode_Vol['KV'].astype(int)
dfX_pivotMode_Vol['SV'] = dfX_pivotMode_Vol['SV'].astype(int)
dfX_pivotMode_Vol['PV'] = dfX_pivotMode_Vol['PV'].astype(int)
dfX_pivotMode_Vol['RV'] = dfX_pivotMode_Vol['RV'].astype(int)
dfX_pivotMode_Vol['TV'] = dfX_pivotMode_Vol['TV'].astype(int)

display(dfX_pivotMode_Vol)

# show numbers with commas
pd.options.display.float_format = '{:,.4f}'.format

dfX_pivotMode_OPct = dfX.pivot_table(index=('group','OID','DID','date_range','day_type','day_part'),values=('OPercentTotal'),columns='mode')
dfX_pivotMode_OPct = dfX_pivotMode_OPct.reset_index()
dfX_pivotMode_OPct = dfX_pivotMode_OPct.fillna(0)
dfX_pivotMode_OPct = dfX_pivotMode_OPct.rename(columns={'OID':'O','DID':'D','ALL':"AO",'BIC':'KO','BUS':'SO','PED':'PO','RAI':'RO','TRU':'TO'})
display(dfX_pivotMode_OPct)

dfX_pivotMode_DPct = dfX.pivot_table(index=('group','OID','DID','date_range','day_type','day_part'),values=('DPercentTotal'),columns='mode')
dfX_pivotMode_DPct = dfX_pivotMode_DPct.reset_index()
dfX_pivotMode_DPct = dfX_pivotMode_DPct.fillna(0)
dfX_pivotMode_DPct = dfX_pivotMode_DPct.rename(columns={'OID':'O','DID':'D','ALL':"AD",'BIC':'KD','BUS':'SD','PED':'PD','RAI':'RD','TRU':'TD'})
display(dfX_pivotMode_DPct)

dfX_pivotMode_Cmb = pd.DataFrame.merge(dfX_pivotMode_Vol, dfX_pivotMode_OPct, on=('group','O','D','date_range','day_type','day_part'), how='left')
dfX_pivotMode_Cmb = pd.DataFrame.merge(dfX_pivotMode_Cmb, dfX_pivotMode_DPct, on=('group','O','D','date_range','day_type','day_part'), how='left')

dfX_pivotMode_Cmb['O'] = dfX_pivotMode_Cmb['O'].apply(lambda x: '{0:0>4}'.format(x))
dfX_pivotMode_Cmb['D'] = dfX_pivotMode_Cmb['D'].apply(lambda x: '{0:0>4}'.format(x))
dfX_pivotMode_Cmb['I'] = dfX_pivotMode_Cmb['O'].astype(str) + '_' + dfX_pivotMode_Cmb['D'].astype(str)

dfX_pivotMode_Cmb = dfX_pivotMode_Cmb.drop(columns=['O','D'])

display(dfX_pivotMode_Cmb)

dfFiles = dfX_pivotMode_Cmb.groupby(['group','date_range','day_type','day_part'],as_index=False).agg(NUMREC=('I','count'))

for index, row in dfFiles.iterrows():
    #print(row['date_range']) 
    dfExport = dfX_pivotMode_Cmb[(dfX_pivotMode_Cmb['date_range']==row['date_range']) & (dfX_pivotMode_Cmb['day_type']==row['day_type']) & (dfX_pivotMode_Cmb['day_part']==row['day_part'])]
    dfExport = dfExport.drop(columns=['group','date_range','day_part','day_type'])
    dfExport = dfExport.set_index('I')
    dfExport.to_json(os.path.join('results', row['group'] + '_' + str(row['date_range']) + '_' + str(row['day_type']) + '_' + str(row['day_part']) + '_wiIntrazonals.json'),orient='index')

mode,group,O,D,date_range,day_type,day_part,AV,KV,SV,PV,RV,TV
0,centertosmalldist,11,2000,2019_03,0,0,1,0,0,0,0,4
1,centertosmalldist,11,2000,2019_03,0,2,1,0,0,0,0,0
2,centertosmalldist,11,2000,2019_03,0,4,0,0,0,0,0,4
3,centertosmalldist,11,2000,2019_03,1,0,0,0,0,0,0,8
4,centertosmalldist,11,2000,2019_03,1,4,0,0,0,0,0,8
...,...,...,...,...,...,...,...,...,...,...,...,...
8008155,smalldist,721,721,2021_YR,2,1,9,1,0,14,0,0
8008156,smalldist,721,721,2021_YR,2,2,150,38,0,472,0,0
8008157,smalldist,721,721,2021_YR,2,3,204,44,0,821,0,0
8008158,smalldist,721,721,2021_YR,2,4,155,22,0,555,0,0


mode,group,O,D,date_range,day_type,day_part,AO,KO,SO,PO,RO,TO
0,centertosmalldist,11,2000,2019_03,0,0,0.0167,0.0000,0.0000,0.0000,0.0000,0.0367
1,centertosmalldist,11,2000,2019_03,0,2,0.0833,0.0000,0.0000,0.0000,0.0000,0.0000
2,centertosmalldist,11,2000,2019_03,0,4,0.0000,0.0000,0.0000,0.0000,0.0000,0.0625
3,centertosmalldist,11,2000,2019_03,1,0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0541
4,centertosmalldist,11,2000,2019_03,1,4,0.0000,0.0000,0.0000,0.0000,0.0000,0.0976
...,...,...,...,...,...,...,...,...,...,...,...,...
8008155,smalldist,721,721,2021_YR,2,1,0.4286,1.0000,0.0000,0.7778,0.0000,0.0000
8008156,smalldist,721,721,2021_YR,2,2,0.6148,0.8261,0.0000,0.9478,0.0000,0.0000
8008157,smalldist,721,721,2021_YR,2,3,0.3475,0.7719,0.0000,0.8243,0.0000,0.0000
8008158,smalldist,721,721,2021_YR,2,4,0.3138,0.7333,0.0000,0.8296,0.0000,0.0000


mode,group,O,D,date_range,day_type,day_part,AD,KD,SD,PD,RD,TD
0,centertosmalldist,11,2000,2019_03,0,0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0002
1,centertosmalldist,11,2000,2019_03,0,2,0.0001,0.0000,0.0000,0.0000,0.0000,0.0000
2,centertosmalldist,11,2000,2019_03,0,4,0.0000,0.0000,0.0000,0.0000,0.0000,0.0011
3,centertosmalldist,11,2000,2019_03,1,0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0003
4,centertosmalldist,11,2000,2019_03,1,4,0.0000,0.0000,0.0000,0.0000,0.0000,0.0015
...,...,...,...,...,...,...,...,...,...,...,...,...
8008155,smalldist,721,721,2021_YR,2,1,0.2432,0.3333,0.0000,0.7000,0.0000,0.0000
8008156,smalldist,721,721,2021_YR,2,2,0.5282,0.7308,0.0000,0.8906,0.0000,0.0000
8008157,smalldist,721,721,2021_YR,2,3,0.3842,0.8627,0.0000,0.8169,0.0000,0.0000
8008158,smalldist,721,721,2021_YR,2,4,0.3799,0.6471,0.0000,0.8525,0.0000,0.0000


mode,group,date_range,day_type,day_part,AV,KV,SV,PV,RV,TV,...,PO,RO,TO,AD,KD,SD,PD,RD,TD,I
0,centertosmalldist,2019_03,0,0,1,0,0,0,0,4,...,0.0000,0.0000,0.0367,0.0000,0.0000,0.0000,0.0000,0.0000,0.0002,0011_2000
1,centertosmalldist,2019_03,0,2,1,0,0,0,0,0,...,0.0000,0.0000,0.0000,0.0001,0.0000,0.0000,0.0000,0.0000,0.0000,0011_2000
2,centertosmalldist,2019_03,0,4,0,0,0,0,0,4,...,0.0000,0.0000,0.0625,0.0000,0.0000,0.0000,0.0000,0.0000,0.0011,0011_2000
3,centertosmalldist,2019_03,1,0,0,0,0,0,0,8,...,0.0000,0.0000,0.0541,0.0000,0.0000,0.0000,0.0000,0.0000,0.0003,0011_2000
4,centertosmalldist,2019_03,1,4,0,0,0,0,0,8,...,0.0000,0.0000,0.0976,0.0000,0.0000,0.0000,0.0000,0.0000,0.0015,0011_2000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8008155,smalldist,2021_YR,2,1,9,1,0,14,0,0,...,0.7778,0.0000,0.0000,0.2432,0.3333,0.0000,0.7000,0.0000,0.0000,0721_0721
8008156,smalldist,2021_YR,2,2,150,38,0,472,0,0,...,0.9478,0.0000,0.0000,0.5282,0.7308,0.0000,0.8906,0.0000,0.0000,0721_0721
8008157,smalldist,2021_YR,2,3,204,44,0,821,0,0,...,0.8243,0.0000,0.0000,0.3842,0.8627,0.0000,0.8169,0.0000,0.0000,0721_0721
8008158,smalldist,2021_YR,2,4,155,22,0,555,0,0,...,0.8296,0.0000,0.0000,0.3799,0.6471,0.0000,0.8525,0.0000,0.0000,0721_0721


In [ ]:
dfOD_wiIntrazonals_Orig_withGroupTotals[(dfOD_wiIntrazonals_Orig_withGroupTotals['OID']==21) & (dfOD_wiIntrazonals_Orig_withGroupTotals['DID']==21)]

,OID,DID,day_type,day_part,odvolume,odtime,mode,date_range,group,OTotalVolume,OPercentTotal
